In [1]:
import nengo
import matplotlib.pyplot as plt
import numpy
import gym
%matplotlib inline

In [2]:
env = gym.make('FrozenLake-v0')
new_state = env.reset()
model = nengo.Network()

[2017-09-01 16:56:26,863] Making new env: FrozenLake-v0


In [4]:
def one_hot(location, size):
    one_hot_vector = numpy.zeros(size)
    one_hot_vector[location] = 1
    return one_hot_vector

[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]


In [ ]:
with model:
    #One Hot representation of current state as given by the environment. 
    state_vector = one_hot(state)
    state_node = nengo.Node(state_vector)
    #Action vectors
    action_vector = numpy.array([0, 1])
    action_node = nengo.Node(action_vector)
    
    #dlpfc keeps a repository of all states, Connecting the one hot representation of state node to dlpfc ensemble with
    #sixteen dimensions
    
    dlpfc_state = nengo.networks.EnsembleArray(n_neurons=1000, n_ensembles=16, radius = 1)
    nengo.Connection(state_node, dlpfc_state.input)
    dlpfc_state_probe = nengo.Probe(dlpfc_state.output)

    dlpfc_action = nengo.Ensemble(n_neurons=100, dimensions=2, radius = 1)
    #nengo.Connection(state_node, dlpfc_action)
    #dlpfc_action_probe = nengo.Probe(dlpfc_action)

    #mOFC infers one state based on previous state and all states information obtained from latOFC with environmental 
    #cues. Observation is the environment cue -- will be noisy eventually. 
    observation = nengo.Node([0, 1])
    latOFC = nengo.Ensemble(n_neurons=1000, dimensions=2, radius = 4)
    prev_state = nengo.Ensemble(n_neurons=1000, dimensions=2, radius = 4)
    #nengo.Connection(mOFC, prev_state)
    
    

In [ ]:
with model:
    state_vector = one_hot(state)
    state_node = nengo.Node(state_vector)
    state = nengo.networks.EnsembleArray(n_neurons=1000, n_ensembles=env.observation_space.n, radius=1)
    nengo.Connection(state_node, state.input)
    

In [ ]:
#Test with simulator and plot
sim = nengo.Simulator(model)
sim.run(0.5)
plt.figure()
plt.plot(sim.trange(), sim.data[dlpfc_state_probe])